## 准备数据

In [14]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [15]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [16]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(shape=[28 * 28, 512], dtype=tf.float32,
            initial_value=tf.random.normal(shape=[28 * 28, 512]))
        self.b1 = tf.Variable(shape=[512], dtype=tf.float32, initial_value=tf.zeros(shape=[512]))

        self.W2 = tf.Variable(shape=[512, 10], dtype=tf.float32,
            initial_value=tf.random.normal(shape=[512, 10]))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(shape=[10]))

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [x.shape[0], -1])
        logits = tf.matmul(tf.nn.relu(tf.matmul(x, self.W1) + self.b1), self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [17]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [18]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 207.4923 ; accuracy 0.104616664
epoch 1 : loss 174.65172 ; accuracy 0.10193333
epoch 2 : loss 156.26036 ; accuracy 0.10405
epoch 3 : loss 143.55434 ; accuracy 0.10898333
epoch 4 : loss 134.03447 ; accuracy 0.11523333
epoch 5 : loss 126.4726 ; accuracy 0.123
epoch 6 : loss 120.24098 ; accuracy 0.13048333
epoch 7 : loss 114.9113 ; accuracy 0.13881667
epoch 8 : loss 110.231094 ; accuracy 0.14845
epoch 9 : loss 106.01011 ; accuracy 0.15816666
epoch 10 : loss 102.11651 ; accuracy 0.16766667
epoch 11 : loss 98.479515 ; accuracy 0.17838334
epoch 12 : loss 95.060684 ; accuracy 0.18933333
epoch 13 : loss 91.8387 ; accuracy 0.2008
epoch 14 : loss 88.79859 ; accuracy 0.21228333
epoch 15 : loss 85.93025 ; accuracy 0.2239
epoch 16 : loss 83.22372 ; accuracy 0.23515
epoch 17 : loss 80.66999 ; accuracy 0.2469
epoch 18 : loss 78.258415 ; accuracy 0.25755
epoch 19 : loss 75.9789 ; accuracy 0.26898333
epoch 20 : loss 73.828125 ; accuracy 0.28013334
epoch 21 : loss 71.79453 ; accuracy 0.29